##SETUP

In [3]:
import os
import time
import pandas as pd
import numpy as np
# Android ADB
from scrcpy import Client, const

# Image processing
from PIL import Image
import cv2
import pytesseract
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm
# internal
import bot_core
import bot_perception

import importlib
importlib.reload(bot_core)
importlib.reload(bot_perception)

bot = bot_core.Bot()
# Evaluate grid to update OCR images
names=bot.scan_grid()
grid_df=bot_perception.grid_status(names)
display(grid_df)

,grid_id,unit,probability,rank,rank_error,position,Age
0,icon_0,crystal.png,185.0,2,605.0,"[0, 0]",0.0
1,icon_1,crystal.png,385.0,3,178.5,"[0, 1]",0.0
2,icon_2,dryad.png,291.0,2,585.0,"[0, 2]",0.0
3,icon_3,monkey.png,556.0,2,610.0,"[0, 3]",0.0
4,icon_4,hunter.png,249.0,3,213.0,"[0, 4]",0.0
5,icon_5,monkey.png,324.0,1,1217.0,"[1, 0]",0.0
6,icon_6,dryad.png,664.0,2,275.0,"[1, 1]",0.0
7,icon_7,empty.png,999.0,0,0.0,"[1, 2]",0.0
8,icon_8,hunter.png,670.0,1,1489.0,"[1, 3]",0.0
9,icon_9,crystal.png,270.0,2,605.0,"[1, 4]",0.0


In [13]:
# Check if alive
bot.client.control.text("test")


b'\x01\x00\x00\x00\x04test'

In [51]:
bot.getScreen()

## Get grid meta

In [4]:
names=bot.scan_grid()
grid_df=bot_perception.grid_status(names)
grid_df=bot_perception.grid_status(names,grid_df)
display(grid_df)
prev_grid= grid_df.copy()

,grid_id,unit,probability,rank,rank_error,position,Age
0,icon_0,crystal.png,185.0,2,605.0,"[0, 0]",1.0
1,icon_1,crystal.png,385.0,3,178.5,"[0, 1]",1.0
2,icon_2,dryad.png,291.0,2,585.0,"[0, 2]",1.0
3,icon_3,monkey.png,556.0,2,610.0,"[0, 3]",1.0
4,icon_4,hunter.png,249.0,3,213.0,"[0, 4]",1.0
5,icon_5,monkey.png,324.0,1,1217.0,"[1, 0]",1.0
6,icon_6,dryad.png,664.0,2,275.0,"[1, 1]",1.0
7,icon_7,empty.png,999.0,0,0.0,"[1, 2]",1.0
8,icon_8,hunter.png,670.0,1,1489.0,"[1, 3]",1.0
9,icon_9,crystal.png,270.0,2,605.0,"[1, 4]",1.0


In [16]:
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
#unit_series.drop(labels='empty.png',level=0)
merge_series = unit_series[unit_series>2]
merge_target =  merge_series.sample().index[0]
merge_df=df_split.get_group(merge_target)
merge_df.sample(n=2)

,grid_id,unit,probability,rank,rank_error,position,Age
2,icon_2,dryad.png,291.0,2,585.0,"[0, 2]",1.0
10,icon_10,dryad.png,319.0,2,580.0,"[2, 0]",1.0


In [82]:
# Returns all elements which match tokens value
def filter_keys(unit_series,tokens,remove=False):
    # Add detection of dimension in input tokens
    merge_series= unit_series
    for level in tokens:
        series= []
        for token in level:
            # check if given token is int, assume unit rank filter
            if isinstance(token,int):
                exists = merge_series.index.get_level_values('rank').isin([token]).any()
                series.append(merge_series.xs(token, level='rank',drop_level=False) if exists else pd.Series(dtype=object))
            else:
                series.append(merge_series.xs(token, level='unit',drop_level=False) if token in merge_series else pd.Series(dtype=object))
        merge_series = pd.concat(series)
    if remove:
        merge_series = unit_series[~unit_series.index.isin(merge_series.index)]
    return merge_series
# Remove tuple tokens from unit series
def remove_keys(unit_series,tokens = [('empty.png', 0)]):
    return unit_series[~unit_series.index.isin(tokens)]

In [76]:
for level in [[3,4,5]]:
    for token in level:
        print(token)

3
4
5


In [85]:
merge_series = unit_series
filter_keys(merge_series,[
    [3,4,5],
    ['crystal.png']
    ],remove=True)

unit         rank
dryad.png    2       3
crystal.png  2       2
empty.png    0       2
crystal.png  1       1
dryad.png    1       1
hunter.png   1       1
             3       1
monkey.png   1       1
             2       1
             3       1
Name: unit, dtype: int64

In [74]:
keep_series = filter_keys(merge_series,[[3,4,5],['crystal.png']],True)


TypeError: unhashable type: 'list'

In [66]:
merge_series = unit_series
# Remove all high level crystals
keep_series = filter_keys(merge_series,[3,4,5])
merge_series = filter_keys(keep_series,['crystal.png'],True)
merge_series

unit        rank
hunter.png  3       1
monkey.png  3       1
dtype: int64

In [61]:
filter_keys(filter_keys(unit_series,[3,4,5]),['crystal.png'],True)


unit        rank
hunter.png  3       1
monkey.png  3       1
dtype: int64

In [34]:
# Remove tuple tokens from unit series
def remove_keys(unit_series,tokens = [('empty.png', 0)]):
    return unit_series[~unit_series.index.isin([tokens])]
merge_series 

array([False,  True, False,  True, False, False, False, False, False,
       False, False])

In [37]:
merge_series[~merge_series.index.isin([('crystal.png', 1),('crystal.png', 2)])]

unit         rank
dryad.png    2       3
empty.png    0       2
crystal.png  3       1
dryad.png    1       1
hunter.png   1       1
             3       1
monkey.png   1       1
             2       1
             3       1
Name: unit, dtype: int64

In [30]:
merge_series[merge_series.index !=('crystal.png', 1)]

unit         rank
dryad.png    2       3
crystal.png  2       2
empty.png    0       2
crystal.png  3       1
dryad.png    1       1
hunter.png   1       1
             3       1
monkey.png   1       1
             2       1
             3       1
Name: unit, dtype: int64

In [29]:
merge_series

unit         rank
dryad.png    2       3
crystal.png  2       2
empty.png    0       2
crystal.png  1       1
             3       1
dryad.png    1       1
hunter.png   1       1
             3       1
monkey.png   1       1
             2       1
             3       1
Name: unit, dtype: int64